In [1]:
using TrajectoryOptimization
using Plots
using MeshCat
using GeometryTypes
using CoordinateTransformations
using FileIO
using LinearAlgebra

┌ Info: Recompiling stale cache file /Users/zacman/.julia/compiled/v1.0/TrajectoryOptimization/Flw6O.ji for TrajectoryOptimization [7d60e499-3ad9-587e-a260-f318c741da60]
└ @ Base loading.jl:1190


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/41d86076dd52b0fc27d8eaaa40add6f6ed466438-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/41d86076dd52b0fc27d8eaaa40add6f6ed466438-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
### Solver options ###
opts = SolverOptions()
opts.square_root = false
opts.verbose = true
# opts.cache=true
# opts.c1=1e-4
# opts.c2 = 2.0
# opts.mu_al_update = 10.0
# opts.eps_constraint = 1e-3
# opts.eps_intermediate = 1e-5
# opts.eps = 1e-5
opts.outer_loop_update = :uniform
opts.τ = 0.1
# opts.iterations_outerloop = 250
# opts.iterations = 1000

0.1

In [3]:
# Model
n = 13 # states (quadrotor w/ quaternions)
m = 4 # controls
model = Model(Dynamics.quadrotor_dynamics!,n,m)

Model(TrajectoryOptimization.Dynamics.quadrotor_dynamics!, 13, 4)

In [4]:
# Objective and constraints
Qf = 100.0*Matrix{Float64}(I, n, n)
Q = (0.1)*Matrix{Float64}(I, n, n)
R = (0.001)*Matrix{Float64}(I, m, m)
tf = 5.0
dt = 0.05

# -initial state
x0 = zeros(n)
x0[4:7] = [1;0;0;0]

# -final state
xf = zeros(n)
xf[1:3] = [20.0;20.0;0.0] # xyz position
xf[4:7] = [1;0;0;0]
xf

# -control limits
u_min = -10.0
u_max = 10.0

# -obstacles
quad_radius = 3.0
sphere_radius = 1.0

n_spheres = 4
spheres = ([5.0;9.0;9.0;15.0;],[5.0;9.0;9.0;15.0],[0.0;0.0;4.0;0.0],[sphere_radius;sphere_radius;sphere_radius;sphere_radius])
function cI(x,u)
    [TrajectoryOptimization.sphere_constraint(x,spheres[1][1],spheres[2][1],spheres[3][1],spheres[4][1]+quad_radius);
     TrajectoryOptimization.sphere_constraint(x,spheres[1][2],spheres[2][2],spheres[3][2],spheres[4][2]+quad_radius);
     TrajectoryOptimization.sphere_constraint(x,spheres[1][3],spheres[2][3],spheres[3][3],spheres[4][3]+quad_radius);
     TrajectoryOptimization.sphere_constraint(x,spheres[1][4],spheres[2][4],spheres[3][4],spheres[4][4]+quad_radius);
     -x[3]]
end

# -constraint that quaternion should be unit
#function cE(x,u)
#    [x[4]^2 + x[5]^2 + x[6]^2 + x[7]^2 - 1.0]
#end

obj_uncon = UnconstrainedObjective(Q, R, Qf, tf, x0, xf)
obj_con = TrajectoryOptimization.ConstrainedObjective(obj_uncon, u_min=u_min, u_max=u_max, cI=cI)
#obj_con = TrajectoryOptimization.ConstrainedObjective(obj_uncon, u_min=u_min, u_max=u_max, cI=cI, cE = cE)

Custom inequality constraints are not inplace
 -converting to inplace
 -THIS IS SLOW


ConstrainedObjective{Array{Float64,2},Array{Float64,2},Array{Float64,2}}([0.1 0.0 … 0.0 0.0; 0.0 0.1 … 0.0 0.0; … ; 0.0 0.0 … 0.1 0.0; 0.0 0.0 … 0.0 0.1], [0.001 0.0 0.0 0.0; 0.0 0.001 0.0 0.0; 0.0 0.0 0.001 0.0; 0.0 0.0 0.0 0.001], [100.0 0.0 … 0.0 0.0; 0.0 100.0 … 0.0 0.0; … ; 0.0 0.0 … 100.0 0.0; 0.0 0.0 … 0.0 100.0], 0.0, 5.0, [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [20.0, 20.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-10.0, -10.0, -10.0, -10.0], [10.0, 10.0, 10.0, 10.0], [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf], [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf], getfield(TrajectoryOptimization, Symbol("#f!#2")){typeof(cI)}(cI), getfield(TrajectoryOptimization, Symbol("##7#11"))(), true, 13, 13, 13, 0)

In [5]:
Dynamics.quadrotor_dynamics(x0, [1;1;1;1])

13×1 Array{Float64,2}:
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
  0.0               
 -1.8100000000000005
  0.0               
  0.0               
  0.0               

In [6]:
# Solver
solver_uncon = Solver(model,obj_uncon,integration=:rk3,dt=dt,opts=opts)
solver = Solver(model,obj_con,integration=:rk3,dt=dt,opts=opts)


# - Initial control and state trajectories
U = ones(solver.model.m, solver.N);
# X_interp = line_trajectory(solver)

In [11]:
### Solve ###
results, stats = solve(solver_uncon,U)
#results, stats = solve(solver,U)

┌ Info: Solving Unconstrained Problem...
└ @ TrajectoryOptimization /Users/zacman/.julia/dev/TrajectoryOptimization/src/ilqr_methods.jl:874
┌ Info: Outer loop 1 (begin)
└ @ TrajectoryOptimization /Users/zacman/.julia/dev/TrajectoryOptimization/src/solve.jl:168
____iter cost          expected    actual      z         α         c_max     dJ        ρ         dgrad     sgrad     grad      j         zero_counterinfo                                              
____-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
____1     716.66397124  69265.0745              0.999128  1                   69204.68  0.625     222.6853  43.14465  7.28545  1         0         
____2     93.912232774  623.696298              0.998486  1                   622.7517  0.244141  39.984    13.86966  0.933551 1         0         
____3     89.871914195  5.92524021           

(UnconstrainedVectorResults(Array{Float64,1}[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.238859, 0.957607, -0.182442, 0.182442, -3.03552e-16, -0.890786, -0.890786, -8.87741, -15.1989, 15.1989, -2.498e-14], [0.235396, 0.235396, -0.368384, 0.814629, -0.392865, 0.392865, -1.10299e-15, 10.8083, 10.8083, 3.15342, -3.60525, 3.60525, -2.49728e-14], [0.928297, 0.928297, -0.159188, 0.797275, -0.408258, 0.408258, -2.02777e-15, 16.8406, 16.8406, 5.41251, 2.0344, -2.0344, -1.66389e-14], [1.76913, 1.76913, 0.0985285, 0.841004, -0.361725, 0.361725, -2.91243e-15, 16.7935, 16.7935, 4.89423, 2.43671, -2.43671, -1.14998e-14], [2.57815, 2.57815, 0.305215, 0.881325, -0.31003, 0.31003, -3.83901e-15, 15.588, 15.588, 3.32628, 2.28751, -2.28751, -1.42609e-14], [3.332, 3.332, 0.428682, 0.915243, -0.256217, 0.256217, -4.75265e-15, 14.5919, 14.5919, 1.57149, 2.42721, -2.42721, -1.79074e-14], [4.0448, 4.0448, 0.46656, 0.944853, -0.195193, 0.195193, -5.59989e-15, 13.9475, 13.94

In [12]:
### Results ###
println("Final position: $(results.X[1:3,end])\n       desired: $(obj_uncon.xf[1:3])\n    Iterations: $(stats["iterations"])\n Max violation: $(max_violation(results.result[results.termination_index]))")
println("Final position (euler): $(results_euler.X[1:3,end])\n       desired: $(obj_uncon_euler.xf[1:3])\n    Iterations: $(stats_euler["iterations"])\n Max violation: $(max_violation(results_euler.result[results_euler.termination_index]))")

ErrorException: type UnconstrainedVectorResults has no field result

In [13]:
# Position
plot(results.X[1:3,:]',color="green",title="Quadrotor Position xyz",xlabel="Time",ylabel="Position",label=["x";"y";"z"])
plot!(results_euler.X[1:3,:]',color="blue")

MethodError: MethodError: Cannot `convert` an object of type Array{Float64,2} to an object of type Float64
Closest candidates are:
  convert(::Type{T<:AbstractFloat}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S} at /Users/zacman/.julia/packages/Ratios/ZrTTx/src/Ratios.jl:16
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  ...

In [10]:
# Control
plot(results.U[1:m,:]',color="green")
plot!(results_euler.U[1:m,:]',color="blue")

UndefVarError: UndefVarError: results not defined

Quaternion Animation

In [14]:
### Visualizer using MeshCat and GeometryTypes ###
# Set up visualizer
vis = Visualizer()
open(vis)

# Import quadrotor obj file
urdf_folder = joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf")
# urdf = joinpath(joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf"), "quadrotor.urdf")
obj = joinpath(joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf"), "quadrotor_base.obj")

# color options
green = MeshPhongMaterial(color=RGBA(0, 1, 0, 1.0))
red = MeshPhongMaterial(color=RGBA(1, 0, 0, 1.0))
blue = MeshPhongMaterial(color=RGBA(0, 0, 1, 1.0))
orange = MeshPhongMaterial(color=RGBA(233/255, 164/255, 16/255, 1.0))
black = MeshPhongMaterial(color=RGBA(0, 0, 0, 1.0))
black_transparent = MeshPhongMaterial(color=RGBA(0, 0, 0, 0.1))

# geometries
robot_obj = load(obj)
sphere_small = HyperSphere(Point3f0(0), convert(Float32,0.1*quad_radius)) # trajectory points
sphere_medium = HyperSphere(Point3f0(0), convert(Float32,quad_radius))

obstacles = vis["obs"]
traj = vis["traj"]
target = vis["target"]
robot = vis["robot"]

# Set camera location
# settransform!(vis["/Cameras/default"], compose(Translation(25., -5., 10),LinearMap(RotZ(-pi/4))))

# Create and place obstacles
for i = 1:n_spheres
    setobject!(vis["obs"]["s$i"],HyperSphere(Point3f0(0), convert(Float32,spheres[4][i])),red)
    settransform!(vis["obs"]["s$i"], Translation(spheres[1][i], spheres[2][i], spheres[3][i]))
end

# Create and place trajectory
for i = 1:solver.N
    setobject!(vis["traj"]["t$i"],sphere_small,blue)
    settransform!(vis["traj"]["t$i"], Translation(results.X[1,i], results.X[2,i], results.X[3,i]))
end

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8700
└ @ MeshCat /Users/zacman/.julia/packages/MeshCat/rZWLp/src/servers.jl:7
┌ Info: Listening on: Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 0x21fc)
└ @ HTTP.Servers /Users/zacman/.julia/packages/HTTP/YjRCz/src/Servers.jl:301
┌ Info: Accept (0):  🔗    0↑     0↓    1s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /Users/zacman/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (1):  🔗    0↑     0↓    1s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /Users/zacman/.julia/packages/HTTP/YjRCz/src/Servers.jl:343


UndefVarError: UndefVarError: Pkg not defined

┌ Info: Accept (2):  🔗    0↑     0↓    0s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /Users/zacman/.julia/packages/HTTP/YjRCz/src/Servers.jl:343


In [ ]:
# Create and place initial position
setobject!(vis["robot"]["ball"],sphere_medium,black_transparent)
setobject!(vis["robot"]["quad"],robot_obj,black)
settransform!(vis["robot"],compose(Translation(results.X[1,1], results.X[2,1], results.X[3,1]),LinearMap(TrajectoryOptimization.quat2rot(results.X[4:7,1]))))

In [ ]:
# Animate quadrotor
for i = 1:solver.N
    settransform!(vis["robot"], compose(Translation(results.X[1,i], results.X[2,i], results.X[3,i]),LinearMap(TrajectoryOptimization.quat2rot(results.X[4:7,i]))))
    sleep(solver.dt*2)
end

# # -unconstrained version
# for i = 1:solver.N
#     settransform!(vis["robot"], compose(Translation(results_uncon.X[1,i], results_uncon_euler.X[2,i], results_uncon.X[3,i]),LinearMap(TrajectoryOptimization.quat2rot(results_uncon.X[4:7,i]))))
#     sleep(solver.dt*2)
# end


Euler Angle Animation

In [ ]:
### Visualizer using MeshCat and GeometryTypes ###
# Set up visualizer
vis = Visualizer()
open(vis)

In [ ]:
# Import quadrotor obj file
urdf_folder = joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf")
# urdf = joinpath(joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf"), "quadrotor.urdf")
obj = joinpath(joinpath(Pkg.dir("TrajectoryOptimization"), "dynamics/urdf"), "quadrotor_base.obj")

# color options
green = MeshPhongMaterial(color=RGBA(0, 1, 0, 1.0))
red = MeshPhongMaterial(color=RGBA(1, 0, 0, 1.0))
blue = MeshPhongMaterial(color=RGBA(0, 0, 1, 1.0))
orange = MeshPhongMaterial(color=RGBA(233/255, 164/255, 16/255, 1.0))
black = MeshPhongMaterial(color=RGBA(0, 0, 0, 1.0))
black_transparent = MeshPhongMaterial(color=RGBA(0, 0, 0, 0.1))

# geometries
robot_obj = load(obj)
sphere_small = HyperSphere(Point3f0(0), convert(Float32,0.1*quad_radius)) # trajectory points
sphere_medium = HyperSphere(Point3f0(0), convert(Float32,quad_radius))

obstacles = vis["obs"]
traj = vis["traj"]
target = vis["target"]
robot = vis["robot"]

# Set camera location
# settransform!(vis["/Cameras/default"], compose(Translation(25., -5., 10),LinearMap(RotZ(-pi/4))))

# Create and place obstacles
for i = 1:n_spheres
    setobject!(vis["obs"]["s$i"],HyperSphere(Point3f0(0), convert(Float32,spheres[4][i])),red)
    settransform!(vis["obs"]["s$i"], Translation(spheres[1][i], spheres[2][i], spheres[3][i]))
end

# Create and place trajectory
for i = 1:solver.N
    setobject!(vis["traj"]["t$i"],sphere_small,blue)
    settransform!(vis["traj"]["t$i"], Translation(results_euler.X[1,i], results_euler.X[2,i], results_euler.X[3,i]))
end

In [ ]:
# Create and place initial position
setobject!(vis["robot"]["ball"],sphere_medium,black_transparent)
setobject!(vis["robot"]["quad"],robot_obj,black)
settransform!(vis["robot"],compose(Translation(results_euler.X[1,1], results_euler.X[2,1], results_euler.X[3,1]),LinearMap(quat2rot(eul2quat(results_euler.X[4:7,1])))))

In [ ]:
# Animate quadrotor
for i = 1:solver.N
    settransform!(vis["robot"], compose(Translation(results_euler.X[1,i], results_euler.X[2,i], results_euler.X[3,i]),LinearMap(quat2rot(eul2quat(results_euler.X[4:7,i])))))
    sleep(solver_euler.dt)
end

# # -unconstrained version
# for i = 1:solver.N
#     settransform!(vis["robot"], compose(Translation(results_uncon_euler.X[1,i], results_uncon_euler.X[2,i], results_uncon_euler.X[3,i]),LinearMap(quat2rot(eul2quat(results_uncon_euler.X[4:7,i])))))
#     sleep(solver_euler.dt*2)
# end

In [ ]:
# Plot Euler angle trajectories
eul = zeros(3,solver.N)
for i = 1:solver.N
#     eul[:,i] = TrajectoryOptimization.quat2eul(results.X[4:7,i])
    eul[:,i] = results_euler.X[4:6,i]

end

plot(eul',title=("Euler angle trajectories"))
